# Song Clusters

In [4]:
import os
import json
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

In [5]:
# define credentials
cid = "XXX"
secret = "XXX"
redirect_uri = "XXX"
username = "XXX"

In [11]:
# Auth
scope = "user-library-read"
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth = token)
else:
    print("Can't get token for " + username)

In [16]:
# Fetch saved songs for the user
saved_tracks_df = pd.DataFrame()
track_list = ""
date_added_list = []
artist_list = []
title_list = []

more_songs = True
offset_index = 0

while more_songs: 
    songs = sp.current_user_saved_tracks(offset = offset_index)
    
    for song in songs["items"]:
        
        # add song ids to track list
        track_list += song["track"]["id"] + ","
        
        # add time when the song was added
        date_added_list.append(song["added_at"])
        
        # add title of the song
        title_list.append(song["track"]["name"])
        
        # add all artists of the song
        artists = song["track"]["artists"]
        artists_name = ""
        for artist in artists:
            artists_name += artist["name"] + ","
        artist_list.append(artists_name[:-1])
    
    # get track features and append them to the dataframe
    track_features = sp.audio_features(track_list[:-1])
    temp_df = pd.DataFrame(track_features)
    saved_tracks_df = saved_tracks_df.append(temp_df)
    track_list = ""
    
    if songs["next"] == None:
        more_songs = False
    else:
        offset_index += songs["limit"]

saved_tracks_df["artists"] = artist_list
saved_tracks_df["song_title"] = title_list
saved_tracks_df["date_added"] = date_added_list

In [17]:
saved_tracks_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artists,song_title,date_added
0,0.746,0.695,4,-7.517,0,0.0400,0.19600,0.026700,0.199,0.8610,...,audio_features,4xymOOdANFwko4RpeR961e,spotify:track:4xymOOdANFwko4RpeR961e,https://api.spotify.com/v1/tracks/4xymOOdANFwk...,https://api.spotify.com/v1/audio-analysis/4xym...,258453,4,"Crystal Castles,HEALTH",Crimewave,2020-03-26T22:30:02Z
1,0.651,0.751,8,-7.422,1,0.3180,0.00982,0.000000,0.266,0.8440,...,audio_features,7i5LIw3UC2WgqIp798GGx6,spotify:track:7i5LIw3UC2WgqIp798GGx6,https://api.spotify.com/v1/tracks/7i5LIw3UC2Wg...,https://api.spotify.com/v1/audio-analysis/7i5L...,195280,4,Jupiter,Starlighter,2020-03-26T21:58:16Z
2,0.556,0.893,10,-3.766,0,0.0621,0.23900,0.000000,0.123,0.2600,...,audio_features,3Q6mwJseOFYBJ10d5CXp4o,spotify:track:3Q6mwJseOFYBJ10d5CXp4o,https://api.spotify.com/v1/tracks/3Q6mwJseOFYB...,https://api.spotify.com/v1/audio-analysis/3Q6m...,253547,4,"Kavinsky,The Weeknd",Odd Look,2020-03-26T21:58:12Z
3,0.777,0.851,10,-3.801,0,0.0844,0.00769,0.000000,0.140,0.6980,...,audio_features,5bzaS5tAni5oUpW7bWVdtY,spotify:track:5bzaS5tAni5oUpW7bWVdtY,https://api.spotify.com/v1/tracks/5bzaS5tAni5o...,https://api.spotify.com/v1/audio-analysis/5bza...,163387,4,"G-Eazy,Gunna",I Wanna Rock (feat. Gunna),2020-03-25T14:24:38Z
4,0.775,0.831,7,-3.897,1,0.2090,0.20000,0.000000,0.208,0.5640,...,audio_features,5LTHlxvc91VFkz4aOSraFQ,spotify:track:5LTHlxvc91VFkz4aOSraFQ,https://api.spotify.com/v1/tracks/5LTHlxvc91VF...,https://api.spotify.com/v1/audio-analysis/5LTH...,161587,4,"French Montana,Juicy J",50's & 100's (feat. Juicy J),2020-03-25T12:33:44Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0.622,0.679,11,-5.621,0,0.2450,0.00275,0.006830,0.606,0.0878,...,audio_features,5jXNOgH5A7yI76BU9kaP6N,spotify:track:5jXNOgH5A7yI76BU9kaP6N,https://api.spotify.com/v1/tracks/5jXNOgH5A7yI...,https://api.spotify.com/v1/audio-analysis/5jXN...,378893,5,"Kanye West,JAY-Z,Rick Ross,Nicki Minaj,Bon Iver",Monster,2017-11-30T18:11:17Z
2,0.763,0.598,11,-6.865,1,0.0539,0.07140,0.000000,0.112,0.1820,...,audio_features,1yxgsra98r3qAtxqiGZPiX,spotify:track:1yxgsra98r3qAtxqiGZPiX,https://api.spotify.com/v1/tracks/1yxgsra98r3q...,https://api.spotify.com/v1/audio-analysis/1yxg...,190678,4,Travis Scott,BUTTERFLY EFFECT,2017-11-24T20:08:09Z
3,0.559,0.840,10,-6.772,0,0.0310,0.03290,0.000005,0.927,0.4720,...,audio_features,7a5bkwWwhbQiB9GZKkwzw6,spotify:track:7a5bkwWwhbQiB9GZKkwzw6,https://api.spotify.com/v1/tracks/7a5bkwWwhbQi...,https://api.spotify.com/v1/audio-analysis/7a5b...,184413,4,Paul Engemann,Scarface (Push It To The Limit),2017-10-26T20:13:12Z
4,0.678,0.770,9,-4.399,0,0.2220,0.00583,0.000000,0.109,0.4120,...,audio_features,158DIbrVt4YbqNnWyRCS3P,spotify:track:158DIbrVt4YbqNnWyRCS3P,https://api.spotify.com/v1/tracks/158DIbrVt4Yb...,https://api.spotify.com/v1/audio-analysis/158D...,284000,4,"Nas,Diddy",Hate Me Now (feat. Puff Daddy),2017-10-26T19:43:09Z


In [19]:
saved_tracks_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'artists', 'song_title', 'date_added'],
      dtype='object')

In [20]:
saved_tracks_df["type"]

0    audio_features
1    audio_features
2    audio_features
3    audio_features
4    audio_features
          ...      
1    audio_features
2    audio_features
3    audio_features
4    audio_features
5    audio_features
Name: type, Length: 606, dtype: object